In [1]:
from datasets import Dataset
import pandas as pd
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoTokenizer, set_seed
import os
from sklearn.model_selection import train_test_split
from scipy.special import softmax
import argparse
import logging
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import wandb
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import os
from torch.utils.data import Dataset
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from datasets import Dataset
import numpy as np
import numpy as np
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

import pandas as pd,os
import torch
from statistics import mode
from sklearn.metrics import confusion_matrix
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer
from datasets import Dataset, load_metric
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
import pickle

c:\Users\Ghiki\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
MODEL1 = 'xlm-roberta-base'
MODEL2 = 'bert-base-multilingual-cased'
MODEL3 = 'bert-base-multilingual-uncased'
MODEL4 = 'intfloat/multilingual-e5-base'

MODEL_PATH1='SavedModels/xlm-roberta-base0k'
MODEL_PATH2='SavedModels/bert-base-multilingual-cased0k'
MODEL_PATH3='SavedModels/bert-base-multilingual-uncased0k'
MODEL_PATH4='SavedModels/multilingual-e5-base0k'

'''Preparing data'''

df = pd.read_json('datasets/subtaskA_dev_multilingual.jsonl', lines=True)

df = df.rename(columns={'source': 'language'})
non_language_sources = ['wikihow', 'wikipedia', 'reddit', 'arxiv', 'peerread']
df['language'] = df['language'].replace(non_language_sources, 'english')

print(f'Original dataset')
print(df.info())

print(f'''\n{df['label'].value_counts()}''')
print(f'''\n{df['model'].value_counts()}''')
print(f'''\n{df['language'].value_counts()}''')

df = df[['text', 'label']]
test_df=df

test_texts = test_df['text'].tolist()

def getPrediction(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    id2label = {0: "human", 1: "machine"}
    label2id = {"human": 0, "machine": 1}
    model = AutoModelForSequenceClassification.from_pretrained(
        model_path, num_labels=len(label2id), id2label=id2label, label2id=label2id
        )

    test_dataset = Dataset.from_pandas(test_df)

    def preprocess_function(examples, **fn_kwargs):
        return fn_kwargs['tokenizer'](examples["text"], truncation=True)

    tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True,  fn_kwargs={'tokenizer': tokenizer})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # create Trainer
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )
    # get logits from predictions and evaluate results using classification report
    predictions = trainer.predict(tokenized_test_dataset)
    preds = np.argmax(predictions.predictions, axis=-1)
    probs = softmax(predictions.predictions, axis=-1)
    label_specific_probs = probs[:, 1]  # This extracts the probability for label 1
    
    return list(preds),list(label_specific_probs)

print(f'\nProcessing with {MODEL1}')
labels1,scores1=getPrediction(MODEL_PATH1)

print(f'\nProcessing with {MODEL2}')
labels2,scores2=getPrediction(MODEL_PATH2)

print(f'\nProcessing with {MODEL3}')
labels3,scores3=getPrediction(MODEL_PATH3)

print(f'\nProcessing with {MODEL4}')
labels4,scores4=getPrediction(MODEL_PATH4)

Original dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      4000 non-null   object
 1   label     4000 non-null   int64 
 2   model     4000 non-null   object
 3   language  4000 non-null   object
 4   id        4000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 156.4+ KB
None

label
1    2000
0    2000
Name: count, dtype: int64

model
human      2000
chatGPT    1500
davinci     500
Name: count, dtype: int64

language
russian    2000
arabic     1000
german     1000
Name: count, dtype: int64
Processing with 
xlm-roberta-base


Map: 100%|██████████| 4000/4000 [00:00<00:00, 5453.38 examples/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 500/500 [00:22<00:00, 22.41it/s]


Processing with n\bert-base-multilingual-cased


Map: 100%|██████████| 4000/4000 [00:00<00:00, 4461.42 examples/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 500/500 [00:23<00:00, 20.97it/s]


Processing with 
bert-base-multilingual-uncased


Map: 100%|██████████| 4000/4000 [00:01<00:00, 3771.37 examples/s]
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 500/500 [00:23<00:00, 21.42it/s]


Processing with 
intfloat/multilingual-e5-base


Map: 100%|██████████| 4000/4000 [00:00<00:00, 4228.30 examples/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 500/500 [00:22<00:00, 22.70it/s]


In [9]:
def getMetrics(predicted_labels, true_labels):
    # Ensure the labels are numpy arrays
    predicted_labels = np.array(predicted_labels)
    true_labels = np.array(true_labels)

    # Compute metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1 = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')[2]
    precision = precision_score(true_labels, predicted_labels, average='binary')
    recall = recall_score(true_labels, predicted_labels, average='binary')
    auc = roc_auc_score(true_labels, predicted_labels)
    cm = confusion_matrix(true_labels, predicted_labels)

    # Create a dictionary of metrics
    metrics = {
        'f1': f1,
        'confusion_matrix': cm.tolist(),
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'auc': auc,
    }

    return metrics

print(MODEL1)
print(getMetrics(labels1,test_df['label'].tolist()))
print(MODEL2)
print(getMetrics(labels2,test_df['label'].tolist()))
print(MODEL3)
print(getMetrics(labels3,test_df['label'].tolist()))
print(MODEL4)
print(getMetrics(labels4,test_df['label'].tolist()))

xlm-roberta-base
{'f1': 0.7522601995214531, 'confusion_matrix': [[1309, 691], [290, 1710]], 'accuracy': 0.75475, 'precision': 0.7122032486463973, 'recall': 0.855, 'auc': 0.75475}
bert-base-multilingual-cased
{'f1': 0.6483303237505995, 'confusion_matrix': [[863, 1137], [191, 1809]], 'accuracy': 0.668, 'precision': 0.6140529531568228, 'recall': 0.9045, 'auc': 0.668}
bert-base-multilingual-uncased
{'f1': 0.6095041681894916, 'confusion_matrix': [[714, 1286], [150, 1850]], 'accuracy': 0.641, 'precision': 0.5899234693877551, 'recall': 0.925, 'auc': 0.641}
intfloat/multilingual-e5-base
{'f1': 0.6443666306473895, 'confusion_matrix': [[816, 1184], [142, 1858]], 'accuracy': 0.6685, 'precision': 0.6107823800131492, 'recall': 0.929, 'auc': 0.6685000000000001}


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

models = [MODEL1, MODEL2, MODEL3, MODEL4]

df = pd.DataFrame({
    f'Labels_{MODEL1}': labels1,
    f'Scores_{MODEL1}': scores1,
    f'Labels_{MODEL2}': labels2,
    f'Scores_{MODEL2}': scores2,
    f'Labels_{MODEL3}': labels3,
    f'Scores_{MODEL3}': scores3,
    f'Labels_{MODEL4}': labels4,
    f'Scores_{MODEL4}': scores4,
})

labels = test_df['label'].tolist()

df.info()

def ensemble_methods(df, models):
    
    majority_labels = []
    score_based_labels = []
    rank_voting_labels = []
    borda_count_labels = []

    for index, row in df.iterrows():
        label_counts = {0: 0, 1: 0}
        score_sums = {0: 0.0, 1: 0.0}
        weighted_scores = {0: 0.0, 1: 0.0}

        for i, model in enumerate(models):
            label = row[f'Labels_{model}']
            score = row[f'Scores_{model}']
            label_counts[label] += 1
            score_sums[label] += score

        # Majority Voting
        majority_label = 0 if label_counts[0] > label_counts[1] else 1
        majority_labels.append(majority_label)

        # Soft Voting
        avg_score_0 = score_sums[0] / (label_counts[0] if label_counts[0] else 1)
        avg_score_1 = score_sums[1] / (label_counts[1] if label_counts[1] else 1)
        score_based_label = 0 if avg_score_0 > avg_score_1 else 1
        score_based_labels.append(score_based_label)
        
        # Rank Voting
        ranks = [row[f'Scores_{model}'] for model in models]
        ranked_labels = [label for _, label in sorted(zip(ranks, [row[f'Labels_{model}'] for model in models]))]
        rank_voting_labels.append(ranked_labels[0])  # The label with the lowest rank

        # Borda Count
        borda_scores = {0: 0, 1: 0}
        for rank, label in enumerate(ranked_labels):
            borda_scores[label] += (len(models) - rank)
        borda_count_labels.append(max(borda_scores, key=borda_scores.get))
        
    return {
        'Majority Voting':majority_labels,
        'Soft Voting':score_based_labels,
        'Rank Voting':rank_voting_labels,
        'Borda Count':borda_count_labels,
    }

ensemble_results = ensemble_methods(df, models)

for voting_method, details in ensemble_results.items():
    print(f'''{voting_method}: {getMetrics(details,labels)['f1']}''')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 8 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Labels_xlm-roberta-base                4000 non-null   int64  
 1   Scores_xlm-roberta-base                4000 non-null   float32
 2   Labels_bert-base-multilingual-cased    4000 non-null   int64  
 3   Scores_bert-base-multilingual-cased    4000 non-null   float32
 4   Labels_bert-base-multilingual-uncased  4000 non-null   int64  
 5   Scores_bert-base-multilingual-uncased  4000 non-null   float32
 6   Labels_intfloat/multilingual-e5-base   4000 non-null   int64  
 7   Scores_intfloat/multilingual-e5-base   4000 non-null   float32
dtypes: float32(4), int64(4)
memory usage: 187.6 KB
Majority Voting: 0.6042970169577611
Soft Voting: 0.501177432773524
Rank Voting: 0.7894884262125959
Borda Count: 0.7208065451583845


In [12]:
import itertools

# Assuming all_models is a list of your model names
all_models = [MODEL1, MODEL2, MODEL3, MODEL4]

# This will store the best F1 score, the corresponding model combination, and the voting method
best_f1_score = 0
best_model_combination = None
best_voting_method = None

# Define the voting methods you want to evaluate
voting_methods = ['Majority Voting', 'Soft Voting', 'Rank Voting', 'Borda Count']

# Try all possible combinations of the models
for r in range(1, len(all_models) + 1):
    for model_combination in itertools.combinations(all_models, r):
        # Generate the predictions using the ensemble of the current combination of models
        ensemble_results = ensemble_methods(df, model_combination)
        
        # Evaluate each voting method
        for method in voting_methods:
            f1_score = getMetrics(ensemble_results[method], labels)['f1']
            
            # Update the best combination if the current one is better
            if f1_score > best_f1_score:
                best_f1_score = f1_score
                best_model_combination = model_combination
                best_voting_method = method

# Print the best combination, its score, and the voting method
print(f"Best F1 Score: {best_f1_score}")
print(f"Best Model Combination: {best_model_combination}")
print(f"Best Voting Method: {best_voting_method}")


Best F1 Score: 0.7894884262125959
Best Model Combination: ('xlm-roberta-base', 'bert-base-multilingual-cased', 'bert-base-multilingual-uncased', 'intfloat/multilingual-e5-base')
Best Voting Method: Rank Voting
